In [2]:
import pandas as pd
import json

In [3]:
#filenameIn = "data/goldsequences-0.json"
filenameIn = "data/goldsequences-0-1-2-3-4-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21-22-23-24-25-26-27-28-29.json"
f = open(filenameIn)
data = json.load(f)
print("Tasks stored in this gold data: " + str(data.keys()))


Tasks stored in this gold data: dict_keys(['0', '1', '2', '3', '4', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29'])


In [4]:
def get_observation_look(text_observation: str, inventory: bool = False) -> list:
    ignore_terms = ['You also see:', '(', ')', 'In it, you see:', 'In your inventory, you see:']
    text = text_observation
    for ignore in ignore_terms:
        text = text.replace(ignore, ' ')
    all_beliefs = [sentence.replace('\t', '') for sentence in text.split('\n') if len(sentence) > 0]
    if inventory:
        return [f"You have {s.lower()} in your inventory" for s in all_beliefs]
    else:
        return all_beliefs

In [5]:
agent_information = []
for taskIdx in data.keys():
    taskData = data[taskIdx]
    taskName = taskData['taskName']
    goldSequences = taskData['goldActionSequences']
    for goldSequence in goldSequences:
        fold = goldSequence['fold']
        path = goldSequence['path']
        for step in path:
            action = step['action']
            observation = step['observation']
            freelook = step['freelook']
            inventory = step['inventory']
            all_obs = get_observation_look(observation)
            all_inventory_obs = get_observation_look(inventory, inventory=True)
            agent_information += all_obs
            agent_information += all_inventory_obs



data_df = pd.DataFrame(agent_information)
data_df.head()

,0
0,This room is called the hallway.
1,a picture
2,a substance called air
3,the agent
4,


In [6]:
len(agent_information), len(list(set(agent_information)))

(2100684, 23110)

In [7]:
def clean_text(text: str) -> str:
    ignore_tokens = [".", "(", ")", ":", ","]
    result = text
    for token in ignore_tokens:
        result = result.replace(token, " ")
    return result.lower()

data_sentence = []

for sentence in list(set(agent_information)):
    preprocessed_sentence = clean_text(sentence)
    data_sentence.append({
        "sentence": preprocessed_sentence,
        "num_words": len(preprocessed_sentence.split())
    })
sentence_df = pd.DataFrame(data_sentence)
sentence_df.head()

,sentence,num_words
0,terminal 2 on unknown substance m is now conne...,15
1,you have chocolate in your inventory,6
2,a table on the table is a green wire a red ...,43
3,a flower pot 1 containing a unknown b plant i...,53
4,an inclined plane with a brass surface with ...,16


In [18]:
sentence_df.sort_values("num_words", ascending=False)

,sentence,num_words
19410,a flower pot 1 containing soil a unknown e p...,78
15050,a flower pot 9 containing soil a unknown e p...,76
15081,a flower pot 4 containing a unknown e plant i...,76
15069,you have a seed jar containing a unknown e pl...,75
2996,a flower pot 1 containing soil a unknown e p...,75
...,...,...
10196,cachew,1
22227,peanut,1
20832,marshmallow,1
12724,bread,1


In [19]:
sentence_df.to_csv("data/dataset_sentence_level.csv", index=False)